In [ ]:
import warnings
warnings.filterwarnings('ignore')
%reload_ext autoreload
%autoreload 2
from vision_ai_api import APIVisionAI
from IPython.display import Markdown
import pandas as pd
import io
import requests
import json

## API


In [ ]:
def get_objects_table_from_sheets(
    url: str = "https://docs.google.com/spreadsheets/d/122uOaPr8YdW5PTzrxSPF-FD0tgco596HqgB7WK7cHFw/edit#gid=1672006844",
):
    request_url = url.replace("edit#gid=", "export?format=csv&gid=")
    response = requests.get(request_url)
    dataframe = pd.read_csv(io.StringIO(response.content.decode("utf-8")), dtype=str)
    dataframe["label"] = dataframe["label"].fillna("null")
    dataframe = dataframe[dataframe["use"] == "1"]
    dataframe = dataframe.drop(columns=["use"])

    return dataframe


df = get_objects_table_from_sheets()
labels_to_put = df.to_dict("records")

In [ ]:
with open("secrets.json") as f:
    s = json.load(f)


vision_ai_api = APIVisionAI(
    username=s.get("username"),
    password=s.get("password"),
)

In [ ]:
objects = vision_ai_api._get_all_pages("/objects")
cameras = vision_ai_api._get_all_pages("/cameras", page_size=3000)
prompts = vision_ai_api._get_all_pages("/prompts")

In [ ]:
## GET A OBJECT Parameters
object_slug = "traffic"
object_parameters = vision_ai_api.get_item_from_slug(slug=object_slug, data=objects)

In [ ]:
## GET ALL active cameras

cameras_ativas = vision_ai_api._get_all_pages(
    "/agents/89173394-ee85-4613-8d2b-b0f860c26b0f/cameras"
)
cameras_ativas_ids = [f"/cameras/{d.get('id')}" for d in cameras_ativas]
cameras_agent = vision_ai_api._get_all_pages(cameras_ativas_ids)

In [ ]:
# #### REMOVE

# objects_to_remove = ["water_level"]
# labels_to_remove = ['puddle','flodding','low_indifferent']
# items_to_remove = []
# # for camera in cameras_agent:
# #     camera_id = camera.get('id')
# #     prompt_id = "e25cabfd-6861-47a8-8ff1-3e3efb34ca57"
# for object_slug in objects_to_remove:
#     for label in labels_to_remove:
#         items_to_remove.append(
#             {
#                 # "camera_id": camera_id,
#                 'object_slug': object_slug,
#                 'label_slug': label,
#                 # "prompt_id": prompt_id
#             }
#         )

# r = vision_ai_api.process_remove_items(
#     items=items_to_remove,
#     objects=objects,
#     cameras=cameras,
#     prompts=prompts
# )

In [ ]:
items_to_remove = [
    {
        "object_slug": "traffic",
        "label_slug": "impossibe",
    }
]

r = vision_ai_api.process_remove_items(
    items=items_to_remove, objects=objects, cameras=cameras, prompts=prompts
)

In [ ]:
# items = df.rename(columns={"label":"label_slug","object":"object_slug"}).to_dict("records")
# items_to_process = []


# for camera in cameras_agent:
#     camera_id = camera.get('id')
#     prompt_id = "e25cabfd-6861-47a8-8ff1-3e3efb34ca57"
#     for item in items:
#         item['camera_id'] = camera_id
#         item['prompt_id'] = prompt_id

#         items_to_process.append(

#                {
#                     "object_slug": item['object_slug'],
#                     # "label_slug": item['label_slug'],
#                     # "criteria": item['criteria'],
#                     # "identification_guide": item['identification_guide'],
#                     "camera_id": camera_id,
#                     "prompt_id": prompt_id
#                 }

#         )

In [ ]:
items = df.rename(columns={"label": "label_slug", "object": "object_slug"})
objects_to_add = list(items["object_slug"].unique())
items_to_process = []
for camera in cameras_agent:
    camera_id = camera.get("id")
    prompt_id = "e25cabfd-6861-47a8-8ff1-3e3efb34ca57"
    for object_slug in objects_to_add:
        items_to_process.append(
            {
                "object_slug": object_slug,
                # "label_slug": item['label_slug'],
                # "criteria": item['criteria'],
                # "identification_guide": item['identification_guide'],
                "camera_id": camera_id,
                # "prompt_id": prompt_id
            }
        )

In [ ]:
items_to_process = [
    {
        "object_slug": "traffic",
        "label_slug": "impossible",
        "criteria": "Complete submergence of the road with high water level",
        "identification_guide": "Identify a scenarios where the road is entirely submerged/flooded, making it completely impassable for vehicles.",
        # "camera_id": camera_id,
        # "prompt_id": prompt_id
    }
]

In [ ]:
vision_ai_api.process_items(
    items=items_to_process,
    objects=objects,
    cameras=cameras,
    # prompts=prompts
)

In [ ]:
# # ### POST CAMERA TO AGENT

# cameras_to_input = [
#     '222',
#     '99',
#     '793',
#     '100',
#     '794',
#     '793',
#     '1613',
#     '1612',
#     '795',
#     '1594',
#     '10',
#     '2',
#     '1624',
#     '108',
#     '1234',
#     '1231',
#     "1175",
#     "1316",
#     "1189",
#     "276",
#     "33",
#     "1472",
#     "3306",
#     "221",
#     "52",
#     "270",
#     "96",
#     "1615",
#     "1623",
#     "211",
#     "1496",
#     "1497",
#     "1390",
#     "211",
#     "102",
#     "362",
#     "432",
#     "260"
# ]

# agent_id = "89173394-ee85-4613-8d2b-b0f860c26b0f"


# for camera_id in cameras_to_input:
#     camera_id = camera_id.zfill(6)
#     r = vision_ai_api._post(
#         path=f'/agents/{agent_id}/cameras?camera_id={camera_id}'
#     )
#     print(r)

In [ ]:
### PUT PROMPT


prompt_text = """

### Role 

- **Urban Road Image Analyst**

### Objective
    
- **Task**: Analyze CCTV images from Rio de Janeiro, specifically focusing on elements related to urban road conditions.

### Input

- **Data Provided**: A CCTV image of an urban road.

### Process

1. **Image Condition Assessment**
    - **Action**: Examine the image for uniform color patches or distortions indicating data loss or image corruption.
    - **Decision**: Assign a label to 'image_condition' reflecting the image's quality based on observed uniform colors, distortions, and blurriness.
    
2. **Detailed Image Description**
    - **Action**: Provide a detailed, unbiased description of the image, focusing on visible aspects for analysis. Minimum word count: 500.
    - **Guideline**: Base your description solely on visible elements in the image, avoiding external assumptions or contexts.

3. **Object Analysis and Classification**
    - **Action**: Use the detailed image description to classify each object per the 'Objects Table'.
    - **Requirement**: Support each classification with clear evidence from the image.

4. **Structured Output Presentation**
    - **Format**: Present findings in a structured JSON format, following the provided example.

### Objects Table 

- **Guidance**: Use the table below for object classification, adhering to the specified criteria and identification guides.

{objects_table_md}


### Output

- **Example Format**: 
        
```json
{output_example}
```

- **Requirement**: Each label explanation should be a 300-word interpretation of the image, demonstrating a deep understanding of the visible elements.

**Output Order**

- **Sequence**: Follow this order in your analysis: 
    1. image_condition
    2. image_description
    3. water_in_road
    4. water_level
    5. traffic_ease_vehicle
    6. road_blockade

- **Importance**: Adhering to this sequence ensures logical and coherent analysis, with each step informing the subsequent ones.

"""


prompt_parameters = vision_ai_api._get("/prompts").get("items")[0]
prompt_id = prompt_parameters.get("id")
request_body = {
    "name": prompt_parameters.get("name"),
    "model": prompt_parameters.get("model"),
    "prompt_text": prompt_text,
    "max_output_token": prompt_parameters.get("max_output_token"),
    "temperature": prompt_parameters.get("temperature"),
    "top_k": prompt_parameters.get("top_k"),
    "top_p": prompt_parameters.get("top_p"),
}


r = vision_ai_api._put(path=f"/prompts/{prompt_id}", json_data=request_body)
print(json.dumps(r.json(), indent=2))

In [ ]:
prompt_parameters = vision_ai_api._get("/prompts").get("items")

In [ ]:
prompt_parameters

In [ ]:
### POST PROMPT


prompt_parameters = vision_ai_api._get("/prompts").get("items")[0]
prompt_id = prompt_parameters.get("id")
request_body = {
    "name": "test",
    "model": prompt_parameters.get("model"),
    "prompt_text": "test",
    "max_output_token": prompt_parameters.get("max_output_token"),
    "temperature": prompt_parameters.get("temperature"),
    "top_k": prompt_parameters.get("top_k"),
    "top_p": prompt_parameters.get("top_p"),
}


r = vision_ai_api._post(path=f"/prompts", json_data=request_body)
print(json.dumps(r.json(), indent=2))